## Portfolio Optimization 

In [20]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import yfinance as yf 
from scipy.optimize import minimize 

In [110]:
### Import File
filepath = '/Users/jasonting/Downloads/Quant Port Data.xlsx'

## Import Tickers
qcom_df = pd.read_excel(filepath, 'QCOM')['Adj Close']
intc_df = pd.read_excel(filepath, 'INTC')['Adj Close']
lmt_df = pd.read_excel(filepath, 'LMT')['Adj Close']
tsm_df = pd.read_excel(filepath, 'TSM')['Adj Close']
bac_df = pd.read_excel(filepath, 'BAC')['Adj Close']
rf = pd.read_excel(filepath, 'RF')['Adj Close']


lc_df = [qcom_df, intc_df, lmt_df,tsm_df, bac_df]



In [64]:
## Combine Data
returns_matrix = pd.concat(lc_df,axis=1)
column_names = ['QCOM', 'INTC', 'LMT', 'TSM', 'BAC']
returns_matrix.columns = column_names
returns_matrix = returns_matrix.pct_change(axis=0).dropna()

print(returns_matrix.head())

       QCOM      INTC       LMT       TSM       BAC
1  0.009213 -0.011891 -0.000709  0.003927 -0.056068
2  0.027985  0.045186  0.056094  0.012256  0.034963
3  0.009581  0.003476 -0.014329 -0.025245  0.024029
4  0.058207  0.027495  0.026332  0.070824  0.002853
5  0.046067  0.008639  0.011632 -0.033317  0.031294


In [97]:
## Generate Initial Weights & Var-Covariance Matrix
initial_weights = w = np.array([1/len(returns_matrix.columns)] * len(returns_matrix.columns))
covariance_matrix = returns_matrix.cov()
print(f'Variance-Covariance Matrix \n\n{covariance_matrix}')

Variance-Covariance Matrix 

          QCOM      INTC       LMT       TSM       BAC
QCOM  0.002585  0.001163  0.000460  0.001430  0.001031
INTC  0.001163  0.002475  0.000539  0.000802  0.001004
LMT   0.000460  0.000539  0.001292  0.000273  0.000675
TSM   0.001430  0.000802  0.000273  0.002114  0.000704
BAC   0.001031  0.001004  0.000675  0.000704  0.002328


In [101]:
## Optimziation (Minimum Variance Portfolio) SCIPY MINIMIZE
def minimum_variance_portfolio(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights


# Constraints
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) -1})

# Bounds
bounds = tuple((0, 1) for asset in range(len(returns_matrix.columns)))

result = minimize(minimum_variance_portfolio, initial_weights, args=(covariance_matrix,), method='SLSQP', bounds=bounds, constraints=constraints)


display(result)

min_variance_weight = result.x


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0008820285188901412
       x: [ 1.432e-02  1.137e-01  5.131e-01  2.106e-01  1.483e-01]
     nit: 6
     jac: [ 1.719e-03  1.785e-03  1.777e-03  1.603e-03  1.937e-03]
    nfev: 36
    njev: 6

array([0.01431782, 0.11368714, 0.51308661, 0.21063551, 0.14827292])

In [113]:
## Optimziation (Minimum Variance Portfolio) Using Gradient Descent Iteration

def calc_port_var(weight,matrix):
    return np.dot(weight.T,np.dot(weight,covariance_matrix))

def partial_deriv(weight, matrix):
    return np.dot(matrix,weight) + np.dot(matrix,weight)

def port_returns(weight,returns_matrix):
    return np.dot(weights, returns_matrix)

matrix = covariance_matrix

iterations = 100000
min_var = []
for n in range(iterations):
    #Gradient vector points in the direction of steepest ascent graphically, we wish to minimize our portfolio variance or find x point that corresponds to the minimum y point of our graph, 

    w = w - partial_deriv(w,matrix)  * .1 #Taken from gradient descent optimization (xold - [gradient(xold)*scalar value]) scalar value adds small increment of movement to x
    excess_w = (1-w.sum())/len(w)
    w = w + excess_w

    #End condition, when iteration stops we want to take the final weights
    if n == iterations-1:        
        min_var.append(w)
        print(f'Minimum Variance Portfolio Weights: {w}')
        print(f'Gradient at this point: {partial_deriv(w,matrix)}')
        print(f'Total Portfolio Weights: {w.sum()}')
        
              

            
return_min = np.dot(min_var[0],returns_matrix.dropna().T).sum()
print(f'Min-Variance Return: {return_min}')




Minimum Variance Portfolio Weights: [0.0124036  0.11061036 0.52167036 0.26499381 0.09032187]
Gradient at this point: [0.00174539 0.00174539 0.00174539 0.00174539 0.00174539]
Total Portfolio Weights: 1.0
Min-Variance Return: 0.9116401964583345


In [119]:
## Highest Sharpe Ratio
def highest_sharpe(weights, returns, cov_matrix, risk_free_rate):
    portfolio_return = np.sum(weights * returns)
    portfolio_std_dev = np.sqrt(weights.T @ cov_matrix @ weights)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return -sharpe_ratio  # Minimize negative Sharpe ratio


result = minimize(highest_sharpe, initial_weights, args=(returns_matrix.mean(), covariance_matrix, .05), method='SLSQP', bounds=bounds, constraints=constraints)

result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.8623059445183392
       x: [ 1.000e+00  1.943e-16  9.159e-16  0.000e+00  3.608e-16]
     nit: 3
     jac: [-9.834e-01 -3.943e-01 -2.003e-01 -6.152e-01 -4.016e-01]
    nfev: 18
    njev: 3